In [1]:
import json
import os
from PIL import Image
import sys
import json
import cv2
import matplotlib.pyplot as plt
import shutil
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Change to a different directory
os.chdir(r'C:\Users\SIU856522160\Downloads\\')
# Get the new default path
os.getcwd()

'C:\\Users\\SIU856522160\\Downloads'

In [2]:
# #### {
#         "name": "b1c66a42-6f7d68ca.jpg",
#         "attributes": {
#             "weather": "overcast",
#             "scene": "city street",
#             "timeofday": "daytime"
#         },
#         "timestamp": 10000,
#         "labels": [
#             {
#                 "category": "traffic sign",
#                 "attributes": {
#                     "occluded": false,
#                     "truncated": false,
#                     "trafficLightColor": "none"
#                 },
#                 "manualShape": true,
#                 "manualAttributes": true,
#                 "box2d": {
#                     "x1": 1000.698742,
#                     "y1": 281.992415,
#                     "x2": 1040.626872,
#                     "y2": 326.91156
#                 },
#                 "id": 0
#             },
            

In [3]:
bdd_annotations_path_train = r".\\bdd100k\labels\bdd100k_labels_images_train.json"
output_file_path = r".\\major\dataset\output\output_train.txt" 

In [4]:
# with open(bdd_annotations_path_train, 'r') as file:
#     annotations = json.load(file)

# # Extract file names from annotations
# file_names = [image_data['name'] for image_data in annotations]

# # Save file names to the output file
# with open(output_file_path, 'w') as output_file:
#     output_file.write('\n'.join(file_names))

# print("File names saved to:", output_file_path)


In [5]:
# with open(r".\\bdd100k\labels\bdd100k_labels_images_val.json", 'r') as file:
#     annotations = json.load(file)

# # Extract file names from annotations
# file_names = [image_data['name'] for image_data in annotations]
# output_file_path = r".\\major\dataset\output\output_val.txt" 
# # Save file names to the output file
# with open(output_file_path, 'w') as output_file:
#     output_file.write('\n'.join(file_names))

# print("File names saved to:", output_file_path)


In [4]:
# Assuming you have loaded the BDD100K dataset annotations into a variable called "annotations"
categories = set()
folder_name = "train"
with open(rf".\\bdd100k\labels\bdd100k_labels_images_{folder_name}.json", 'r') as f:
        bdd_annotations_train = json.load(f)
for annotation in bdd_annotations_train:
        labels = annotation['labels']
        for label in labels:
                category = label['category']
                categories.add(category)

total_categories = len(categories)
print(categories)
print("Total categories:", total_categories)


{'lane', 'drivable area', 'car', 'person', 'bike', 'traffic light', 'rider', 'train', 'truck', 'bus', 'traffic sign', 'motor'}
Total categories: 12


In [17]:
# Assuming you have loaded the BDD100K dataset annotations into a variable called "annotations"
categories = set()
folder_name = "val"
with open(rf".\\bdd100k\labels\bdd100k_labels_images_{folder_name}.json", 'r') as f:
        bdd_annotations_valid = json.load(f)
for annotation in bdd_annotations_valid:
    labels = annotation['labels']
    for label in labels:
        category = label['category']
        categories.add(category)

total_categories = len(categories)
print(categories)
print("Total categories:", total_categories)


In [8]:
# import os

# def create_file_with_filenames(folder_name):
#     folder_path = r"C:\Users\siu856522160\Downloads\100k\test"  # Replace with the actual path to the "val" folder
#     file_path = os.path.join(r".\\major\output\\", f"output_{folder_name}.txt")
    
#     file_names = os.listdir(folder_path)
#     file_names = [file_name for file_name in file_names if os.path.isfile(os.path.join(folder_path, file_name))]
    
#     with open(file_path, "w") as file:
#         file.write("\n".join(file_names))
    
#     print(f"File with file names created: {file_path}")

# # Call the function with folder_name set to "val"
# create_file_with_filenames("test")


In [9]:
#labelling 
category_mapping = {
    "person": 0,
    "rider": 1,
    "car": 2,
    "truck": 3,
    "bus": 4,
    "train": 5,
    "motor": 6,
    "bike": 7,
    "traffic light": 8,
    "traffic sign": 9
}
# {'lane', 'drivable area', 'car', 'person', 'bike', 'traffic light', 'rider', 'train', 'truck', 'bus', 'traffic sign', 'motor'}

In [10]:
def BDD100K_convert(folder_name):
    # Iterate over BDD100K annotations
    if "train" in folder_name.lower() or "val" in folder_name.lower():
        folder_name = folder_name.lower()
        print("Folder is either train or val:", folder_name)
    else:
        print("Folder is not train or val")

    # Load BDD100K annotations
    with open(rf".\\bdd100k\labels\bdd100k_labels_images_{folder_name}.json", 'r') as f:
        bdd_annotations = json.load(f)

    for annotation in bdd_annotations:
        image_name = annotation['name']
        labels = annotation['labels']
        print("Image Name:", image_name)

        # Load the image to retrieve width and height
        image_path = os.path.join(rf".\\100k\{folder_name}", image_name)
        with Image.open(image_path) as img:
            image_width, image_height = img.size
            
            dw = 1.0 / image_width
            dh = 1.0 / image_height

            # Create the YOLOv5 label file
            label_file_path = os.path.join(".\\100k", folder_name, os.path.splitext(image_name)[0] + '.txt')
            with open(label_file_path, 'w') as label_file:
                for label in labels:
                    category = label['category']
                    print("Category:", category)

                    if category in category_mapping:
                        category = category_mapping[category]

                    if 'box2d' not in label:
                        print(label)
                        continue

                    x1 = label['box2d']['x1']
                    y1 = label['box2d']['y1']
                    x2 = label['box2d']['x2']
                    y2 = label['box2d']['y2']

                    x_center = format((x1 + x2) / (2 * image_width), ".6f")
                    y_center = format((y1 + y2) / (2 * image_height), ".6f")
                    width = format((x2 - x1) / image_width, ".6f")
                    height = format((y2 - y1) / image_height, ".6f")
                    
                    xmin = label['box2d']['x1']
                    ymin = label['box2d']['y1']
                    xmax = label['box2d']['x2']
                    ymax = label['box2d']['y2']

                    x = (xmin + xmax) / 2
                    y = (ymin + ymax) / 2
                    w = xmax - xmin
                    h = ymax - ymin

                    x *= dw
                    w *= dw
                    y *= dh
                    h *= dh

                    # Write the label in YOLOv5 format to the label file
                    label_file.write(f"{category} {x_center} {y_center} {width} {height}\n")
                    print(f"{category} {x_center} {y_center} {width} {height}\n")
                    print(f"Formatted x: {x:.6f}, Formatted y: {y:.6f}, Formatted w: {w:.6f}, Formatted h: {h:.6f}")

    

In [11]:
# BDD100K_convert("val")

In [12]:
# !pip install --upgrade pillow matplotlib seaborn


In [13]:

with open('.\\major\logs\output_of_BDD100K_convert_train.txt', 'w') as file:
    # Redirect the standard output to the file
    sys.stdout = file
    
    BDD100K_convert("train")

    # Reset the standard output to the console
    sys.stdout = sys.__stdout__

In [14]:
with open('.\\major\logs\output_of_BDD100K_convert_val.txt', 'w') as file:
    # Redirect the standard output to the file
    sys.stdout = file
    
    BDD100K_convert("val")

    # Reset the standard output to the console
    sys.stdout = sys.__stdout__